# Demand Forecaster Demo

This notebook demonstrates how to use the `demand_forecaster` package for demand forecasting.

We will cover:
1. Loading sales & event data
2. Running daily, weekly, and monthly forecasts
3. Incorporating promotions and discounts
4. Handling events/holidays
5. Attribute-based forecasting (Region, Store, Item)
6. Visualizing forecasts
7. Running forecasts from CLI


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from demand_forecaster.forecaster import AttributeAwareForecaster


## 1. Load Data
We'll use dummy sales and event datasets provided in `examples/`.


In [ ]:
sales = pd.read_csv("examples/sales_data.csv", parse_dates=["date"])
events = pd.read_csv("examples/events.csv", parse_dates=["start_date", "end_date"])

display(sales.head())
display(events.head())

## 2. Initialize Forecaster
We’ll configure horizon frequency (daily/weekly/monthly), lookback windows, and attributes.

In [ ]:
forecaster = AttributeAwareForecaster(
    method="wma",                  # weighted moving average (or "median")
    attributes=["region", "store", "item"],
    horizon_freq="D",              # D = daily, W = weekly, M = monthly
    lookback_config={"D": 8, "W": 8, "M": 5},
    use_trends=True,
    use_promotions=True,
    use_events=True
)

forecaster.fit(sales_df=sales, events_df=events)

## 3. Forecasting - Daily Horizon

In [ ]:
forecast_daily = forecaster.forecast(horizon_periods=14)  # 14 days ahead
forecast_daily.head()

## 4. Forecasting - Weekly Horizon

In [ ]:
forecaster.horizon_freq = "W"
forecast_weekly = forecaster.forecast(horizon_periods=8)
forecast_weekly.head()

## 5. Forecasting - Monthly Horizon

In [ ]:
forecaster.horizon_freq = "M"
forecast_monthly = forecaster.forecast(horizon_periods=6)
forecast_monthly.head()

## 6. Visualization
Let's visualize actual vs forecasted sales.

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(sales.groupby("date")["sales"].sum(), label="Actual")
plt.plot(forecast_daily.set_index("date")["forecast"], label="Forecast (Daily)")
plt.legend()
plt.title("Actual vs Forecasted Sales")
plt.show()

## 7. Using CLI
You can also run forecasts directly from the terminal:

```bash
demand_forecaster run \
  --sales examples/sales_data.csv \
  --events examples/events.csv \
  --freq D \
  --horizon 14 \
  --attributes region store item
```

This will generate forecasts and save them to `forecast_output.csv` by default.